In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import time

In [2]:
orders = pd.read_csv('order_brush_order.csv.zip', compression='zip', 
                                 header=0, sep=',', quotechar='"')

In [3]:
orders.shape

(222750, 4)

In [4]:
orders.head(20)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
5,31119725718155,173811070,116451780,2019-12-27 12:22:05
6,31151322178251,107921853,166741763,2019-12-27 21:08:43
7,31079856153738,178400128,61272835,2019-12-27 01:17:37
8,31118426867571,147941492,10986763,2019-12-27 12:00:27
9,31133458226149,164933170,135957741,2019-12-27 16:10:59


In [5]:
orders['event_time'] = pd.to_datetime(orders['event_time'])  

In [6]:
orders['shopid'].value_counts()

147941492    11703
61556313      6691
54615708      2640
91799978       707
57189823       687
             ...  
158235164        1
147997211        1
166898182        1
194982850        1
189530112        1
Name: shopid, Length: 18770, dtype: int64

In [7]:
group_shops = orders.groupby('shopid')

In [ ]:
group_shops.groups

In [ ]:
group_shops.groups.keys()

In [ ]:
for (shopid, group) in orders.groupby('shopid'):
    print(group.head(5))

In [ ]:
for (shopid, group) in orders.groupby('shopid'):
    print(group.head(5))

In [ ]:
shopid_10159 = group_shops.get_group(181009364)
shopid_10159

In [ ]:
shopid_10159['timestamp'] = shopid_10159["event_time"].values.astype(np.int64) // 10 ** 9
shopid_10159

In [ ]:
shopid_10159.sort_values(by=['timestamp'], inplace=True)
shopid_10159 = shopid_10159.reset_index(drop = True)
shopid_10159

In [ ]:
# df_order_proportion = pd.DataFrame(columns = ["index","order_proportion"]) #建立一個空的 dataframe

# # 用類似 window 的方式，把每筆資料加1小時後的區段皆做檢測
# for i in range(shopid_10159["timestamp"].size):
#     flag = shopid_10159.at[i,"timestamp"] # flag 代表當下挑的那筆資料，往後再加3600秒
#     print("flag: " + str(flag))
#     mask = (shopid_10159["timestamp"] >= flag) & (shopid_10159["timestamp"] <= (flag+3600))
#     df_small = shopid_10159.loc[mask]
#     print(df_small)
    
    
#     number_of_orders = df_small.shape[0]
#     print("number_of_orders: " + str(number_of_orders))
#     number_of_unique_buyers = df_small.userid.nunique()
#     print("number_of_unique_buyers: " + str(number_of_unique_buyers))
#     print("\n")
#     concentrate_rate = number_of_orders/number_of_unique_buyers
        
#     if (concentrate_rate >= 3): # 代表這一段資料裡有 order brushing
#         small_order_proportion = (df_small["userid"].value_counts()/df_small.shape[0]).to_frame(name = "order_proportion")
#         small_order_proportion.reset_index(level=0, inplace=True)
#         df_order_proportion = pd.concat([df_order_proportion, small_order_proportion], axis=0)

In [ ]:
df_order_proportion

In [ ]:
df_result = pd.DataFrame(columns = ["shopid","userid"]) # 這行code要放外層迴圈外面！！！

df_order_brushing = pd.DataFrame(columns = ["orderid","shopid","userid","event_time","timestamp"]) #建立一個空的 dataframe

# 用類似 window 的方式，把每筆資料加1小時後的區段皆做檢測
for i in range(shopid_10159["timestamp"].size):
    flag = shopid_10159.at[i,"timestamp"] # flag 代表當下挑的那筆資料，往後再加3600秒
    print("flag: " + str(flag))
    mask = (shopid_10159["timestamp"] >= flag) & (shopid_10159["timestamp"] <= (flag+3600))
    df_small = shopid_10159.loc[mask]
    print(df_small)
    
    
    number_of_orders = df_small.shape[0]
    print("number_of_orders: " + str(number_of_orders))
    number_of_unique_buyers = df_small.userid.nunique()
    print("number_of_unique_buyers: " + str(number_of_unique_buyers))
    print("\n")
    concentrate_rate = number_of_orders/number_of_unique_buyers
        
    if (concentrate_rate >= 3): # 代表這一段資料裡有 order brushing
        df_order_brushing = pd.concat([df_order_brushing, df_small], axis=0)

        
# calculate the concentrate rate
total_brushing = df_order_brushing.shape[0] # total number of orders during all order brushing periods
df_order_proportion = (df_order_brushing["userid"].value_counts()/total_brushing).to_frame(name = "order_proportion")
df_order_proportion.reset_index(level=0, inplace=True)
df_order_proportion.columns =['userid', 'order_proportion']

# find out which user is the suspicious buyer (having higest order proportion)
max_value = df_order_proportion['order_proportion'].max()
#use a list to save the suspicious buyers
max_order_brushing_list = df_order_proportion[df_order_proportion['order_proportion'] == max_value].userid.tolist()
max_order_brushing_list.sort() # 讓這些 userid 照大小排好

# concat 到 df_output 上面
if(len(max_order_brushing_list)==0):
    df_result = df.append({'shopid': shopid, 'userid':0}, ignore_index=True)

else if(len(max_order_brushing_list)==1):
    userid_result = max_order_brushing_list[0]
    df_result = df.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)

else:
    userid_result = '&'
    userid_result = userid_result.join(map(str, max_order_brushing_list)) #先轉換成 string 後，再用&把每個元素連在一起
    df_result = df.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)



In [ ]:
df_order_brushing

In [ ]:
df_order_proportion

In [ ]:
max_value = df_order_proportion['order_proportion'].max()
#用一個list把這些洗評分的使用者存起來
max_order_brushing_list = df_order_proportion[df_order_proportion['order_proportion'] == max_value].userid.tolist()
max_order_brushing_list.sort() # 讓這些 userid 照大小排好
print(max_order_brushing_list)

In [ ]:
userid_result = '&'
userid_result = userid_result.join(map(str, max_order_brushing_list))
print(userid_result)

In [ ]:
df_order_proportion = (shopid_10159["userid"].value_counts()/shopid_10159.shape[0]).to_frame(name = "order_proportion")
df_order_proportion.reset_index(level=0, inplace=True)
df_order_proportion

In [14]:
orders.shopid.nunique()

18770

## 來把全部組合在一起

In [ ]:
df_result = pd.DataFrame(columns = ["shopid","userid"]) #先建立一個空的DataFrame來裝最後要輸出的結果


##########################################
# 用來計算進度的
total_shopid = orders.shopid.nunique() #18770
x = 1
##########################################

for (shopid, group) in orders.groupby('shopid'):

    df_order_brushing = pd.DataFrame(columns = ["orderid","shopid","userid","event_time","timestamp"]) #建立一個空的 dataframe

    group['timestamp'] = group["event_time"].values.astype(np.int64) // 10 ** 9 # 轉換成timestamp（全部轉換成多少秒）
    group.sort_values(by=['timestamp'], inplace=True)
    group = group.reset_index(drop = True)

    # 用類似 window 的方式，把每筆資料加1小時後的區段皆做檢測
    for i in range(group["timestamp"].size):
        flag = group.at[i,"timestamp"] # flag 代表當下挑的那筆資料，往後再加3600秒
#         print("flag: " + str(flag))
        mask = (group["timestamp"] >= flag) & (group["timestamp"] <= (flag+3600))
        df_small = group.loc[mask]
#         print(df_small)


        number_of_orders = df_small.shape[0]
#         print("number_of_orders: " + str(number_of_orders))
        number_of_unique_buyers = df_small.userid.nunique()
#         print("number_of_unique_buyers: " + str(number_of_unique_buyers))
#         print("\n")
        concentrate_rate = number_of_orders/number_of_unique_buyers

        if (concentrate_rate >= 3): # 代表這一段資料裡有 order brushing
            df_order_brushing = pd.concat([df_order_brushing, df_small], axis=0)


    # calculate the concentrate rate
    total_brushing = df_order_brushing.shape[0] # total number of orders during all order brushing periods
    df_order_proportion = (df_order_brushing["userid"].value_counts()/total_brushing).to_frame(name = "order_proportion")
    df_order_proportion.reset_index(level=0, inplace=True)
    df_order_proportion.columns =['userid', 'order_proportion']

    # find out which user is the suspicious buyer (having higest order proportion)
    max_value = df_order_proportion['order_proportion'].max()
    #use a list to save the suspicious buyers
    max_order_brushing_list = df_order_proportion[df_order_proportion['order_proportion'] == max_value].userid.tolist()
    max_order_brushing_list.sort() # 讓這些 userid 照數字大小排好（題目要求）

    # concat 到 df_result 上面
    if(len(max_order_brushing_list)==0):
        df_result = df_result.append({'shopid': shopid, 'userid':0}, ignore_index=True)
        print("Progress {:2.1%}".format(x / total_shopid), end="\r")
        x = x+1

    elif(len(max_order_brushing_list)==1):
        userid_result = max_order_brushing_list[0]
        df_result = df_result.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)
        print("Progress {:2.1%}".format(x / total_shopid), end="\r")
        x = x+1
    
    else:
        userid_result = '&'
        userid_result = userid_result.join(map(str, max_order_brushing_list)) #先轉換成 string 後，再用&把每個元素連在一起
        df_result = df_result.append({'shopid': shopid, 'userid': userid_result}, ignore_index=True)
        print("Progress {:2.1%}".format(x / total_shopid), end="\r")
        x = x+1


/Users/leosu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/leosu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [9]:
import os
print("The file will be saved under this path: ", os.getcwd())
export_csv = df_result.to_csv (r'submission_' + time.strftime("%Y-%m-%d %H%M%S", time.localtime()) + '.csv', 
                             index = None, header = True) 

The file will be saved under this path:  /Users/leosu/Desktop/Shopee_Code_League/competition/01_Order_Brushing


In [13]:
# 測試用
test_group = group_shops.get_group(54257623)
test_group['timestamp'] = test_group["event_time"].values.astype(np.int64) // 10 ** 9
test_group.sort_values(by=['timestamp'], inplace=True)
test_group = test_group.reset_index(drop = True)
test_group

/Users/leosu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/leosu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,orderid,shopid,userid,event_time,timestamp
0,31081933517728,54257623,214225272,2019-12-27 01:52:13,1577411533
1,31107219109280,54257623,124403451,2019-12-27 08:53:40,1577436820
2,31117274907795,54257623,8162630,2019-12-27 11:41:14,1577446874
3,31128405551315,54257623,100324582,2019-12-27 14:46:45,1577458005
4,31138805737422,54257623,160812408,2019-12-27 17:40:05,1577468405
5,31150565336189,54257623,605530,2019-12-27 20:56:06,1577480166
6,31190270965933,54257623,186990808,2019-12-28 07:57:50,1577519870
7,31190528134435,54257623,186990808,2019-12-28 08:02:08,1577520128
8,31193768550510,54257623,42443121,2019-12-28 08:56:08,1577523368
9,31195591719971,54257623,153003255,2019-12-28 09:26:31,1577525191
